<a href="https://colab.research.google.com/github/Himanshu-Singh11/Colab/blob/main/car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#car Price Prediction

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

df = pd.read_csv("/content/car_dataset.csv")

df = df[df['Price'] != 'Ask For Price'].dropna()
df['Price'] = df['Price'].str.replace(',', '').astype(int)
df['kms_driven'] = df['kms_driven'].str.replace(' kms', '').str.replace(',', '')
df['kms_driven'] = df['kms_driven'].replace('', np.nan).dropna().astype(int)
df['year'] = df['year'].astype(int)

X = df[['company', 'year', 'kms_driven', 'fuel_type']]
y = df['Price']

categorical = ['company', 'fuel_type']
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
], remainder='passthrough')

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
joblib.dump(model, 'car_price_model.pkl')

def predict_price(company, year, kms, fuel_type):
    new_data = pd.DataFrame([[company, year, kms, fuel_type]],
                            columns=['company', 'year', 'kms_driven', 'fuel_type'])
    base_price = model.predict(new_data)[0]

    if year >= 2022:
        base_price *= 1.80
    elif year >= 2020:
        base_price *= 1.50
    elif year >= 2015:
        base_price *= 1.20
    elif year <= 2010:
        base_price *= 0.80

    if kms >= 100000:
        base_price *= 0.70
    elif kms >= 70000:
        base_price *= 0.80
    elif kms >= 50000:
        base_price *= 0.90

    return int(base_price)

company = input("Company of Car: ").strip()
car_name = input("Car Name: ").strip()

try:
    year = int(input("Year of Purchase: ").strip())
    kms = int(input("Kilometers Driven: ").strip())
except:
    print("Invalid number format for year or kilometers.")
    exit()

fuel_type = input("Fuel Type (Petrol/Diesel/CNG): ").strip()

try:
    price = predict_price(company, year, kms, fuel_type)
    print(f"\n Estimated Price for '{company} {car_name}' ({year}): ₹{price:,}")
except:
    print(" Prediction failed. Please check your inputs.")


Company of Car: Mahindra
Car Name: Scorpio
Year of Purchase: 2022
Kilometers Driven: 11000
Fuel Type (Petrol/Diesel/CNG): Petrol

 Estimated Price for 'Mahindra Scorpio' (2022): ₹887,390
